# 심리상담 데이터 셋을 SentenceBERT 모델로 변환

In [5]:
import pandas as pd
from sentence_transformers import SentenceTransformer       # pip install sentence-transformers
from sklearn.metrics.pairwise import cosine_similarity

### SentenceBERT 모델 로드

In [6]:
# 한국어 문장을 768 개의 벡터로 표시해주는 모델
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

In [7]:
sentences = ['안녕하세요?', '한국어 문장 임베딩을 위한 버트 모델입니다.']
embeddings = model.encode(sentences)
print(embeddings)

[[-0.37510428 -0.7733841   0.592771   ...  0.57923526  0.32683456
  -0.6508967 ]
 [-0.09361677 -0.18191528 -0.19230838 ... -0.03165809  0.30412525
  -0.26793608]]


In [8]:
embeddings.shape

(2, 768)

### 데이터셋 
- 웰니스 대화 스크립트 데이터셋
- AI Hub 사이트, 세브란스 정신과

In [9]:
from google.colab import files
up = files.upload()

Saving wellness_dataset_original.csv to wellness_dataset_original.csv


In [13]:
df = pd.read_csv('wellness_dataset_original.csv')
df.head()

,구분,유저,챗봇,Unnamed: 3
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,NaN
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,NaN
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",NaN
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN,NaN


In [14]:
df.shape

(5231, 4)

- 전처리

In [15]:
df = df.drop(columns=['Unnamed: 3'])
df.tail()

,구분,유저,챗봇
5226,현재상태/증상지속,한숨 자고 일어나면 괜찮으려나? 해서 자고 일어났는데도 똑같아요.,NaN
5227,현재상태/증상지속,자고 나면 괜찮을 줄 알았는데 비슷해…,NaN
5228,현재상태/증상지속,지금도 상태가 계속 안 좋아.,NaN
5229,현재상태/증상지속,이게 제 마음과 상관없이 증상이 계속 나타나요.,NaN
5230,현재상태/증상지속,나이 들면 고쳐질 줄 알았는데…아직까지 계속 그래요.,NaN


In [17]:
# 챗봇 대화가 없는 행은 삭제(결측치 제거)
df.dropna(inplace=True)
df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.


In [18]:
df.shape

(1034, 3)

- 인코딩
    - 사용자 발화만 인코딩
    - 추후에 사용자가 입력한 문장과 코사인 유사도를 통해 비슷한 문장을 추출

In [19]:
model.encode(df.유저[0])

array([-4.80606407e-01, -2.94869334e-01,  4.37900126e-01, -6.40137792e-01,
        3.28670405e-02, -3.42647463e-01, -5.47482446e-02,  1.73054393e-02,
       -4.08220738e-01, -5.06033957e-01, -1.68733329e-01, -3.98676932e-01,
       -1.24776766e-01, -9.71540213e-02, -1.65286466e-01,  5.72604174e-03,
        6.13692813e-02, -1.91312209e-01,  2.53917307e-01, -5.85019529e-01,
       -2.84425914e-01, -2.32034922e-01, -3.27080786e-01,  6.72979727e-02,
       -1.62592951e-05, -4.72336262e-01, -3.60021651e-01,  2.91879803e-01,
       -6.63861454e-01, -3.10574591e-01,  5.79524994e-01, -3.11722964e-01,
        1.47700887e-02, -2.12172598e-01,  2.22058088e-01, -1.73828989e-01,
       -3.78458321e-01, -4.20398176e-01, -2.38218814e-01,  6.38704747e-02,
       -1.15304269e-01, -2.44563907e-01, -5.00228405e-01,  1.68355212e-01,
       -6.58360183e-01, -8.91942143e-01, -6.26957059e-01, -3.21965516e-01,
       -7.05358446e-01,  3.71447325e-01, -5.45803368e-01,  7.76299387e-02,
        1.09864458e-01,  

- 데이터 프레임에 embedding 컬럼을 만들어 인코딩된 값을 저장

In [20]:
df['embedding'] = pd.Series([[]] * len(df))
df.shape

(1034, 4)

In [21]:
df['embedding'] = df['유저'].map(lambda x: list(model.encode(x)))
df.head(3)

,구분,유저,챗봇,embedding
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.4806064, -0.29486933, 0.43790013, -0.64013..."
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561574, -0.1450627, 0.2949032, -0.6739495..."
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.66520005, -0.081268415, 1.0945566, 0.10579..."


In [27]:
df.to_csv('wellness_dataset.csv', index=False)

### 간단한 챗봇 테스트

In [22]:
text = '요즘 머리가 아프고 너무 힘들어'
embedding = model.encode(text)
embedding.shape

(768,)

In [23]:
# squeeze()는 리스트 형태의 값을 스칼라 값으로 변환해 줌
df['유사도'] = df.embedding.map(lambda x: cosine_similarity([embedding], [x]).squeeze())
df.head()

,구분,유저,챗봇,embedding,유사도
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.4806064, -0.29486933, 0.43790013, -0.64013...",0.448967
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561574, -0.1450627, 0.2949032, -0.6739495...",0.490199
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.66520005, -0.081268415, 1.0945566, 0.10579...",0.352131
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.7679057, 0.46520728, 0.528507, -0.50760454...",0.422284
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277737, -0.37413877, 0.040532045, -0.862...",0.315118


In [24]:
df.유사도.idxmax()

4871

In [25]:
answer = df.loc[df.유사도.idxmax()]
answer

구분                                                      증상/편두통
유저                            요즘은 머리가 한쪽만 지그시 누르는 것처럼 무겁고 아파요.
챗봇                           으으, 머리가 아프면 정말 힘들죠. 그 마음 정말 이해해요.
embedding    [-0.6085821, 0.06865981, 0.43937862, -0.217735...
유사도                                                   0.829629
Name: 4871, dtype: object

In [26]:
# 답변할 내용
answer.챗봇

'으으, 머리가 아프면 정말 힘들죠. 그 마음 정말 이해해요.'

### 저장한 파일을 확인

In [28]:
df = pd.read_csv('wellness_dataset.csv')
df.head(3)

,구분,유저,챗봇,embedding
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.4806064, -0.29486933, 0.43790013, -0.64013..."
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561574, -0.1450627, 0.2949032, -0.6739495..."
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.66520005, -0.081268415, 1.0945566, 0.10579..."


In [29]:
df.tail(3)

,구분,유저,챗봇,embedding
1031,현재상태/증상악화,지금 상태가 너무 안 좋아서 학교 안 나가고 있어요.,상태가 더 안 좋아지셨군요. 걱정이 되네요.,"[-0.6814687, -0.11450219, 0.03839703, -0.28655..."
1032,현재상태/증상악화,진짜 심해진 거 같긴 해요.,정말 힘드시겠어요. 지금도 증상이 심하신가요?,"[-0.18133017, -0.109809026, -0.21448648, -0.44..."
1033,현재상태/증상악화,그런데 증상이 나빠진 거 같아.,너무 심하시면 병원을 다시 가보는 건 어떨까요?,"[-0.530171, -0.15990521, 0.49804154, -0.003134..."


In [31]:
df.embedding[0][0:20]   # df.embedding 열은 문자열

'[-0.4806064, -0.2948'

In [32]:
# 문자열을 숫자로 변환
import json
x = json.loads(df.embedding[0])

In [33]:
x[0]

-0.4806064

In [34]:
df.embedding = df.embedding.apply(json.loads)
df.embedding[0][0]

-0.4806064

### 사용자 입력 --> 결과 출력

In [35]:
user_input = '요즘 머리가 아프고 너무 힘들어'
embedding = model.encode(user_input)
df['유사도'] = df.embedding.map(lambda x: cosine_similarity([embedding], [x]).squeeze())
answer = df.loc[df.유사도.idxmax()]
result = {
    'category': answer.구분, 'user': user_input, 'chatbot': answer.챗봇, 'similarity': answer.유사도
}
result

{'category': '증상/편두통',
 'user': '요즘 머리가 아프고 너무 힘들어',
 'chatbot': '으으, 머리가 아프면 정말 힘들죠. 그 마음 정말 이해해요.',
 'similarity': 0.8296288965041261}

In [36]:
json.dumps(result)

'{"category": "\\uc99d\\uc0c1/\\ud3b8\\ub450\\ud1b5", "user": "\\uc694\\uc998 \\uba38\\ub9ac\\uac00 \\uc544\\ud504\\uace0 \\ub108\\ubb34 \\ud798\\ub4e4\\uc5b4", "chatbot": "\\uc73c\\uc73c, \\uba38\\ub9ac\\uac00 \\uc544\\ud504\\uba74 \\uc815\\ub9d0 \\ud798\\ub4e4\\uc8e0. \\uadf8 \\ub9c8\\uc74c \\uc815\\ub9d0 \\uc774\\ud574\\ud574\\uc694.", "similarity": 0.8296288965041261}'